Ensure you are using the preinstalled Jupyter kernel `amcinsights`

# Selling Partner Reporting
Authenticate with Selling Partner Auth before using

#### -----------------------
## Global Configs
Run this cell to initialize the notebook interface

In [ ]:
import json
import re
import boto3
import os
from dotenv import load_dotenv
load_dotenv()

boto3_session = boto3.Session()
lambda_client = boto3_session.client('lambda')

invoke_reporting_lambda_name = os.environ['INVOKE_SP_REPORT_SM_NAME']
schedule_sp_report_lambda_name = os.environ['SCHEDULE_SP_REPORT_NAME']

In [ ]:
# (Optional) If multi-credential secret configured, Auth Id to use for API requests.
# If you did not configure your secret for multiple credentials, leave as None.
auth_id = None

#### -----------------------
## Reporting
The following cells show how to request a report from the Selling Partner Reporting API. 
<br />
Refer to the [Developer Docs](https://developer-docs.amazon.com/sp-api/docs/welcome) for additional information on how to [structure your requests](https://developer-docs.amazon.com/sp-api/docs/reports-api-v2021-06-30-tutorial-request-a-report), [select the right region](https://developer-docs.amazon.com/sp-api/docs/sp-api-endpoints), or find your [Marketplace Id](https://developer-docs.amazon.com/sp-api/docs/marketplace-ids).
<br />
<br />
To start, initialize the cell below with the `region` you want to submit the report request to.
<br />
<br />
There is also an optional parameter `table_prefix` that can be included. Left out, the data lake will default to using the provided {Region}-{Report Type} as the destination table prefix.

In [ ]:
# REQUIRED
region = "North America"

# OPTIONAL
table_prefix = "TestPrefix"

### Create Report Request


In [ ]:
# REQUIRED
request_body = {
    "dataStartTime": "2023-12-31",
    "dataEndTime": "2024-01-06",
    "reportType": "GET_BRAND_ANALYTICS_MARKET_BASKET_REPORT",
    "reportOptions": {
            "reportPeriod": "WEEK"
        },
    "marketplaceIds": [
        "ATVPDKIKX0DER"
    ]
}

In [ ]:
create_report_request = {
    "region": region,
    "requestBody": request_body,
    "authId": auth_id,
    "tablePrefix": table_prefix
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_reporting_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_report_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

### Schedule Report Request

CRON format: https://docs.aws.amazon.com/scheduler/latest/UserGuide/schedule-types.html#cron-based

In [ ]:
# REQUIRED
report_request = {
    "requestBody": {
        "dataStartTime": "TODAY(-6)",
        "dataEndTime": "TODAY()",
        "reportType": "GET_BRAND_ANALYTICS_MARKET_BASKET_REPORT",
        "reportOptions": {
                "reportPeriod": "WEEK"
            },
        "marketplaceIds": [
            "ATVPDKIKX0DER"
        ]
    },
    "region": region,
    "authId": auth_id
}
schedule_expression = "cron(0 23 ? * 7 *)"
rule_name = "testrule"

# OPTIONAL
rule_description = "Trigger GET_BRAND_ANALYTICS_MARKET_BASKET_REPORT every Sunday at 11pm"

In [ ]:
create_schedule_request = {
    "report_request": report_request,
    "schedule_expression": schedule_expression,
    "rule_name": rule_name,
    "rule_description": rule_description
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=schedule_sp_report_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_schedule_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    response_message = response.get('Payload').read().decode('UTF-8')
    response_message = response_message.strip('"')
    print(f"Event Rule Url: {response_message}")

----
Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0